In [1]:
import pandas as pd
import os
from pathlib import Path
import toolkitEngineering
import importlib
importlib.reload(toolkitEngineering)

<module 'toolkitEngineering' from 'd:\\Projetos\\ETL-ML-Dengue\\src\\engineering\\toolkitEngineering.py'>

In [2]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_ibge = os.path.join(diretorio_bronze, 'ibge')
diretorio_ibge_csv = os.path.join(diretorio_ibge, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_ibge_pop_silver = os.path.join(diretorio_silver, 'ibge_pop')
diretorio_ibge_pop_tsv_silver = os.path.join(diretorio_ibge_pop_silver, 'tsv')


## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)

# Bronze
os.makedirs(os.path.join(diretorio_bronze), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge_csv), exist_ok=True)

# Silver
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge_pop_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge_pop_tsv_silver), exist_ok=True)



In [3]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_ibge_csv) if arquivo.endswith("csv")]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]

In [38]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    df = pd.read_csv(os.path.join(diretorio_ibge_csv, arquivo_csv), sep=',')
    df['MUNIC_RES'] = df['MUNIC_RES'].apply(lambda x: str(x)[:-1] if len (str(x)) == 7 else str(x))
    df.columns = df.columns.str.lower()
    df.to_csv(os.path.join(diretorio_ibge_pop_tsv_silver, arquivo_tsv), sep='\t', index=False)

In [4]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitEngineering.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_ibge_pop_tsv_silver, arquivo_tsv)
    toolkitEngineering.inserir_no_banco(cursor, conn, caminho_arquivo, 'ibge_pop', delimiter='\t', diretorio_dotenv=diretorio_dotenv)
